In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-2yj4872p
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-2yj4872p
  Resolved https://github.com/facebookresearch/detectron2.git to commit 9604f5995cc628619f0e4fd913453b4d7d61db3f
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 42.9 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp31

In [ ]:
import cv2
import os
import numpy as np
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import pandas as pd
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor

import pandas as pd
from glob import glob
from tqdm import tqdm
import torch

In [ ]:
!pip install -q map-boxes
from map_boxes import mean_average_precision_for_boxes

Couldn't import fast version of function compute_overlap, will use slow one. Check cython intallation


In [ ]:
test_image_folder = "/content/drive/MyDrive/Endoscope Detection/colonoscopy/Dataset/images/test"
img_paths = [os.path.join(test_image_folder, img) for img in os.listdir(test_image_folder) if img.endswith('.png')]
output_data = []

In [ ]:
cfg = get_cfg()
cfg.merge_from_file("/content/drive/MyDrive/Endoscope Detection/colonoscopy/config.yaml")
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/Endoscope Detection/colonoscopy/Detectron2/experiment/v5/model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.0
cfg.MODEL.ROI_HEADS.MAX_DETECTIONS = 500
# cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.5
cfg.MODEL.DEVICE = "cuda"

In [ ]:
predictor = DefaultPredictor(cfg)

/usr/local/lib/python3.11/dist-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=torch.device("cpu"))

In [ ]:
def format_pred(labels, boxes, scores):
    pred_strings = []
    for label, score, bbox in zip(labels, scores, boxes):
        xmin, ymin, xmax, ymax = bbox.astype(np.float32)
        pred_strings.append(f"{label} {score:.4f} {xmin:.1f} {ymin:.1f} {xmax:.1f} {ymax:.1f}")
    return " ".join(pred_strings) if pred_strings else ""

In [ ]:
for img_path in img_paths:
    img = cv2.imread(img_path)
    outputs = predictor(img)

    instances = outputs["instances"].to("cpu")
    boxes = instances.pred_boxes.tensor.numpy()
    scores = instances.scores.numpy()
    labels = instances.pred_classes.numpy().astype(int)

    image_id = os.path.basename(img_path)

    pred_str = format_pred(labels, boxes, scores)

    output_data.append([image_id, pred_str])
df = pd.DataFrame(output_data, columns=["image_id", "PredictionString"])

/usr/local/lib/python3.11/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
df

,image_id,PredictionString
0,2_1_00138.png,1 0.8208 100.4 165.4 164.0 211.8 1 0.3787 204....
1,2_1_02382.png,0 0.9773 29.6 111.4 164.8 228.4 0 0.8001 9.2 6...
2,2_1_00787.png,0 0.9490 127.9 47.3 256.0 227.4 0 0.1590 162.5...
3,2_1_01456.png,0 0.9427 23.6 46.7 146.1 213.8 0 0.6817 52.6 5...
4,2_1_00555.png,0 0.9465 62.5 76.4 168.9 164.5 2 0.1830 68.3 8...
...,...,...
395,2_3_09079.png,2 0.9932 139.4 62.9 222.4 184.8 0 0.0179 135.2...
396,2_3_08095.png,2 0.9986 95.0 61.0 246.5 168.6 0 0.0370 79.5 5...
397,2_3_09554.png,2 0.9815 83.5 141.3 186.8 220.8 1 0.1915 80.5 ...
398,2_3_08743.png,2 0.9950 93.9 95.9 161.6 155.3 0 0.0030 92.9 9...


In [ ]:
submission_det = df

In [ ]:
meta_data = pd.read_csv("/content/drive/MyDrive/Endoscope Detection/colonoscopy/resize_colonoscopy.csv")

In [ ]:
meta_data

,image_id,height,width,organ,lesion_label,location,shape_type_polygon,polygon_mask,bounding_box_shape,x_min,y_min,x_max,y_max,lesion
0,2_1_00001.png,256,256,colonoscopy,0,1,polygon,"1484.52,775.16;1516.82,710.56;1519.30,640.99;1...",rectangle,104.82,18.63,189.91,100.93,ulcer
1,2_1_00001.png,256,256,colonoscopy,0,1,polygon,"1663.40,1478.26;1653.47,1426.09;1616.20,1388.8...",rectangle,163.83,165.53,207.93,233.54,ulcer
2,2_1_00002.png,256,256,colonoscopy,0,1,polygon,"687.01,223.60;687.01,273.29;704.40,337.89;726....",rectangle,85.88,21.12,198.92,183.23,ulcer
3,2_1_00002.png,256,256,colonoscopy,0,1,polygon,"826.14,1200.00;878.31,1207.45;932.97,1187.58;9...",rectangle,61.03,54.35,125.94,150.93,ulcer
4,2_1_00002.png,256,256,colonoscopy,0,1,polygon,"575.20,1038.51;520.55,1050.93;468.37,1050.93;4...",rectangle,26.56,90.68,75.94,131.37,ulcer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2491,2_2_05111.png,256,256,colonoscopy,1,1,polygon,"1113.11,963.83;1070.55,976.60;1032.26,995.74;1...",rectangle,112.28,118.62,186.74,190.96,polyp
2492,2_2_05112.png,256,256,colonoscopy,1,1,polygon,"1230.13,1397.87;1187.57,1408.51;1138.64,1425.5...",rectangle,108.29,171.28,227.70,236.17,polyp
2493,2_2_05113.png,256,256,colonoscopy,1,1,polygon,"1021.62,642.55;979.06,653.19;938.64,668.09;898...",rectangle,101.37,80.32,158.55,129.79,polyp
2494,2_2_05114.png,256,256,colonoscopy,1,1,polygon,"883.32,972.34;845.02,997.87;806.72,1025.53;776...",rectangle,96.59,121.28,140.20,152.93,polyp


In [ ]:
def string_decoder(predict_string):
    s = predict_string.split(' ')
    class_id = [x for x in (s[0::6])]
    score = [x for x in (s[1::6])]
    x_min = [x for x in (s[2::6])]
    y_min = [x for x in (s[3::6])]
    x_max = [x for x in (s[4::6])]
    y_max = [x for x in (s[5::6])]

    return class_id, score, x_min, y_min, x_max, y_max

In [ ]:
from tqdm import tqdm

In [ ]:
submission_list = []

for i, row in tqdm(submission_det.iterrows(), total=len(submission_det)):
    image_id = row['image_id']
    string_decoded = string_decoder(row['PredictionString'])

    for  (class_id, score, xmin, ymin, xmax, ymax) in zip(*string_decoded):
        submission_list.append(
            {
                'image_id': image_id,
                'lesion_label': int(class_id),
                'score': float(score),
                'x_min': float(xmin),
                'y_min': float(ymin),
                'x_max': float(xmax),
                'y_max': float(ymax),
                }
        )

submission_metric = pd.DataFrame(submission_list, columns=['image_id', 'lesion_label', 'score','x_min','y_min', 'x_max', 'y_max'])

100%|██████████| 400/400 [00:00<00:00, 4073.66it/s]


In [ ]:
type(meta_data)

pandas.core.frame.DataFrame

In [ ]:
test_id = [x.split('/')[-1] for x in glob('/content/drive/MyDrive/Endoscope Detection/colonoscopy/Dataset/images/test/*.png')]
test_df = meta_data[meta_data['image_id'].isin(test_id)]
test_df = test_df.sort_values(by='image_id').reset_index(drop=True)

In [ ]:
ann = test_df.copy()
det = submission_metric.copy()

ann = ann[['image_id','lesion_label','x_min','x_max','y_min','y_max']].values

det = det[['image_id','lesion_label', 'score','x_min','x_max','y_min','y_max']].values


mean_ap_50, average_precisions_50 = mean_average_precision_for_boxes(ann, det, iou_threshold=0.5)
print("_________________________________________________________")
mean_ap_75, average_precisions_75 = mean_average_precision_for_boxes(ann, det, iou_threshold=0.75)

Number of files in annotations: 400
Number of files in predictions: 400
Unique classes: 3
Detections length: 400
Annotations length: 400
0                              | 0.411379 |     180
1                              | 0.655510 |     116
2                              | 0.792207 |     200
mAP: 0.619699
_________________________________________________________
Number of files in annotations: 400
Number of files in predictions: 400
Unique classes: 3
Detections length: 400
Annotations length: 400
0                              | 0.029104 |     180
1                              | 0.211799 |     116
2                              | 0.324242 |     200
mAP: 0.188382
